In [1]:
import pandas as pd
import numpy as np  # 표준편차 계산용 모듈
import pymysql          # mysql 입력용 모듈

# 1. Data Manufacturing

In [13]:
bdf = pd.read_csv('./dummyData/testdata.csv',encoding='cp949')
bdf = bdf[['candel_date_time', 'open', 'high', 'low', 'close', 'volume']]
bdf

,candel_date_time,open,high,low,close,volume
0,2023-01-01 16:00:00,21113000.0,21132000.0,21059000.0,21090000.0,93.315245
1,2023-01-01 17:00:00,21086000.0,21122000.0,21076000.0,21079000.0,67.316058
2,2023-01-01 18:00:00,21080000.0,21093000.0,21070000.0,21079000.0,28.540299
3,2023-01-01 19:00:00,21079000.0,21097000.0,21062000.0,21083000.0,41.528268
4,2023-01-01 20:00:00,21083000.0,21091000.0,21058000.0,21070000.0,39.659074
...,...,...,...,...,...,...
1195,2023-02-20 11:00:00,31436000.0,31497000.0,31312000.0,31469000.0,440.991947
1196,2023-02-20 12:00:00,31469000.0,31595000.0,31425000.0,31570000.0,327.507823
1197,2023-02-20 13:00:00,31570000.0,31680000.0,31554000.0,31619000.0,280.676325
1198,2023-02-20 14:00:00,31603000.0,31640000.0,31520000.0,31582000.0,219.422516


## 1-1. MA(Moving Average)

In [14]:
# 종가기준 15시간 이동평균선
def mac(ndf):
    ndf['MA_C15H'] = 0
    csum = 0
    n = 0
    for i in range(14,len(ndf),1):
        # if i > 997: #check point
        #     print(i)
        for j in range(0+n,15+n,1):
            csum = csum + ndf['close'][j]
        ma = int(csum / 15)
        ndf['MA_C15H'][i] = ma
        csum = 0
        n = n + 1
    return ndf

In [17]:
madf = pd.read_csv('test_ma.csv',encoding='cp949')
madf

,candel_date_time,open,high,low,close,volume,MA_C15H
0,2023-01-01 16:00:00,21113000.0,21132000.0,21059000.0,21090000.0,93.315245,0
1,2023-01-01 17:00:00,21086000.0,21122000.0,21076000.0,21079000.0,67.316058,0
2,2023-01-01 18:00:00,21080000.0,21093000.0,21070000.0,21079000.0,28.540299,0
3,2023-01-01 19:00:00,21079000.0,21097000.0,21062000.0,21083000.0,41.528268,0
4,2023-01-01 20:00:00,21083000.0,21091000.0,21058000.0,21070000.0,39.659074,0
...,...,...,...,...,...,...,...
1195,2023-02-20 11:00:00,31436000.0,31497000.0,31312000.0,31469000.0,440.991947,31692200
1196,2023-02-20 12:00:00,31469000.0,31595000.0,31425000.0,31570000.0,327.507823,31675000
1197,2023-02-20 13:00:00,31570000.0,31680000.0,31554000.0,31619000.0,280.676325,31661200
1198,2023-02-20 14:00:00,31603000.0,31640000.0,31520000.0,31582000.0,219.422516,31641466


## 1-2. MACD(Moving Average Convergence Divergence)

In [24]:
# MACD : 12일 지수이동평균 - 26일 지수이동평균
# 시그널 : MACD의 9일 지수이동평균
# 오실레이터 : MACD값 - 시그널값
def macd(ndf):
    ndf['MACD'] = 0
    i = 0
    for l in range(199, len(ndf), 1): # 623
        #print('date :',ndf['candel_date_time'][l]) #check point
        closeprice = pd.DataFrame(columns=['close'])    
        for n in range(0+i,200+i,1):
            closeprice.loc[len(closeprice)] = [ndf['close'][n]]
        #closeprice = closeprice.iloc[::-1]
        # 지수가중평균함수 ewm 적용 한 뒤 평균함수 meam 적용    
        avg12 = closeprice.ewm(span=12, adjust=False).mean()
        avg26 = closeprice.ewm(span=26, adjust=False).mean()
        macd = avg12 - avg26
        ndf['MACD'][l] = int(round(macd['close'][199],-2))
        i = i + 1
    return ndf

In [27]:
macddf = pd.read_csv('test_macd.csv',encoding='cp949')
macddf

,candel_date_time,open,high,low,close,volume,MACD
0,2023-01-09 23:00:00,21744000.0,21793000.0,21732000.0,21761000.0,157.940462,47900
1,2023-01-10 00:00:00,21765000.0,21808000.0,21714000.0,21740000.0,183.162011,44700
2,2023-01-10 01:00:00,21740000.0,21789000.0,21734000.0,21760000.0,124.840834,43300
3,2023-01-10 02:00:00,21763000.0,21813000.0,21745000.0,21771000.0,91.956559,42500
4,2023-01-10 03:00:00,21790000.0,21820000.0,21764000.0,21819000.0,58.359607,45300
...,...,...,...,...,...,...,...
996,2023-02-20 11:00:00,31436000.0,31497000.0,31312000.0,31469000.0,440.991947,-89300
997,2023-02-20 12:00:00,31469000.0,31595000.0,31425000.0,31570000.0,327.507823,-85000
998,2023-02-20 13:00:00,31570000.0,31680000.0,31554000.0,31619000.0,280.676325,-76800
999,2023-02-20 14:00:00,31603000.0,31640000.0,31520000.0,31582000.0,219.422516,-72400


## 1-3. RSI(Relative Strength Index)

In [28]:
# RSI 계산공식
# 가격이 전일 가격보다 상승한 날의 상승분은 U(up) 값이라고 하고,
# 가격이 전일 가격보다 하락한 날의 하락분은 D(down) 값이라고 한다.
# U값과 D값의 평균값을 구하여 그것을 각각 AU(average ups)와 AD(average downs)라 한다.
# AU를 AD값으로 나눈 것을 RS(relative strength) 값이라고 한다. RS 값이 크다는 것은 일정 기간 하락한 폭보다 상승한 폭이 크다는 것을 의미한다.
# RSI = AU / (AU + AD)
# RSI는 15일 기준, 가격은 종가를 기준으로 한다.

def rsi(ndf):
    ndf['RSI'] = 0
    period = 14
    k = 0
    for n in range(199,len(ndf),1):
        print('date : ',ndf['candel_date_time'][n])
        closeprice = pd.DataFrame(columns=['close'])
        for m in range(0+k,200+k,1):
            closeprice.loc[len(closeprice)] = ndf['close'][m]
        #closeprice = closeprice.reindex(index=closeprice.index[::-1]).reset_index()
        closeprice['close'] = closeprice['close']
        delta = closeprice['close'].diff()
        gains, declines = delta.copy(), delta.copy()
        gains[gains < 0] = 0
        declines[declines > 0] = 0
        _gain = gains.ewm(com=(period - 1), min_periods=period).mean()
        _loss = declines.abs().ewm(com=(period - 1), min_periods=period).mean()
        RS = _gain / _loss
        RSI = pd.Series(100 - (100 / (1 + RS)), name="RSI").iloc[-1]
        ndf['RSI'][n] = round(RSI,1) 
        k = k + 1
    return ndf

In [31]:
rsidf = pd.read_csv('test_rsi.csv',encoding='cp949')
rsidf

,candel_date_time,open,high,low,close,volume,RSI
0,2023-01-01 16:00:00,21113000.0,21132000.0,21059000.0,21090000.0,93.315245,0.0
1,2023-01-01 17:00:00,21086000.0,21122000.0,21076000.0,21079000.0,67.316058,0.0
2,2023-01-01 18:00:00,21080000.0,21093000.0,21070000.0,21079000.0,28.540299,0.0
3,2023-01-01 19:00:00,21079000.0,21097000.0,21062000.0,21083000.0,41.528268,0.0
4,2023-01-01 20:00:00,21083000.0,21091000.0,21058000.0,21070000.0,39.659074,0.0
...,...,...,...,...,...,...,...
1195,2023-02-20 11:00:00,31436000.0,31497000.0,31312000.0,31469000.0,440.991947,40.9
1196,2023-02-20 12:00:00,31469000.0,31595000.0,31425000.0,31570000.0,327.507823,45.4
1197,2023-02-20 13:00:00,31570000.0,31680000.0,31554000.0,31619000.0,280.676325,47.5
1198,2023-02-20 14:00:00,31603000.0,31640000.0,31520000.0,31582000.0,219.422516,46.1


## 1-4. BB(Bollinger Bands)

In [41]:
# BB 계산공식
# 종가의 20일 이동평균선(MA)을 계산합니다.
# 종가의 20일 표준편차(SD)를 계산합니다.
# 상한선을 계산합니다: 상한선 = MA + (2 x 표준편차)
# 하한선을 계산합니다: 하한선 = MA - (2 x 표준편차)
# 종가기준으로 계산
def bb(ndf):
    ndf['BB+'] = 0
    ndf['BBC'] = 0
    ndf['BB-'] = 0
    # 표준편차 곱
    unit = 2
    k = 0
    for i in range(199,len(ndf),1):
        # print('date : ',ndf['candel_date_time'][i]) # check point
        closeprice = pd.DataFrame(columns=['close'])
        for j in range(0+k,200+k,1):
            closeprice.loc[len(closeprice)] = ndf['close'][j]
        
        band_std = unit * np.std(closeprice[len(closeprice) - 20:len(closeprice)])
        band_center = np.mean(closeprice[len(closeprice) - 20:len(closeprice)])
        band_plus = band_center + band_std
        band_minus = band_center - band_std
        ndf['BB+'][i] = round(band_plus)
        ndf['BBC'][i] = round(band_center)
        ndf['BB-'][i] = round(band_minus)
        k = k + 1
    return ndf

In [40]:
bbdf = pd.read_csv('test_bb.csv',encoding='cp949')
bbdf

,candel_date_time,open,high,low,close,volume,BB+,BBC,BB-
0,2023-01-01 16:00:00,21113000.0,21132000.0,21059000.0,21090000.0,93.315245,0.0,0.0,0.0
1,2023-01-01 17:00:00,21086000.0,21122000.0,21076000.0,21079000.0,67.316058,0.0,0.0,0.0
2,2023-01-01 18:00:00,21080000.0,21093000.0,21070000.0,21079000.0,28.540299,0.0,0.0,0.0
3,2023-01-01 19:00:00,21079000.0,21097000.0,21062000.0,21083000.0,41.528268,0.0,0.0,0.0
4,2023-01-01 20:00:00,21083000.0,21091000.0,21058000.0,21070000.0,39.659074,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1195,2023-02-20 11:00:00,31436000.0,31497000.0,31312000.0,31469000.0,440.991947,32099977.0,31722400.0,31344823.0
1196,2023-02-20 12:00:00,31469000.0,31595000.0,31425000.0,31570000.0,327.507823,32082488.0,31706800.0,31331112.0
1197,2023-02-20 13:00:00,31570000.0,31680000.0,31554000.0,31619000.0,280.676325,32069830.0,31696450.0,31323070.0
1198,2023-02-20 14:00:00,31603000.0,31640000.0,31520000.0,31582000.0,219.422516,32063785.0,31688050.0,31312315.0


## 1-5. Make Data

In [ ]:
tdf = bdf.copy()
tdf = bb(tdf)
tdf

In [39]:
tdf.to_csv('test_bb.csv', index=False, encoding='cp949')

# 2. Data Preprocessing

In [ ]:
bdf = pd.read_csv('./dummyData/dummydata.csv',encoding='cp949')
bdf.head()

,open,high,low,close,volume,candel_date_time
0,8892000.0,8900000.0,8818000.0,8844000.0,187.461975,2020-04-07 20:00:00
1,8844000.0,8880000.0,8839000.0,8839000.0,96.131647,2020-04-07 20:30:00
2,8839000.0,8910000.0,8828000.0,8853000.0,280.787491,2020-04-07 21:00:00
3,8853000.0,8854000.0,8771000.0,8789000.0,263.368590,2020-04-07 21:30:00
4,8790000.0,8822000.0,8783000.0,8800000.0,165.317984,2020-04-07 22:00:00


In [ ]:
df = []
bdf.columns
for i in range(1,len(bdf.columns), 1):
    text = str(bdf.columns[i])
    ndf = bdf[['candel_date_time',text]]
    df.append(ndf)

In [ ]:
df[2].head()

,candel_date_time,low
0,2020-04-07 20:00:00,8818000.0
1,2020-04-07 20:30:00,8839000.0
2,2020-04-07 21:00:00,8828000.0
3,2020-04-07 21:30:00,8771000.0
4,2020-04-07 22:00:00,8783000.0


In [ ]:
for j in range(0,len(df),1):
    df[j].to_csv(str(bdf.columns[j])+'.csv',index=False,encoding='cp949')

# 3.Data input to MYSQL

In [ ]:
#mysql 모듈 설치
%pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Connect DB(DataBase)
# DB info
DB_HOST = '127.0.0.1'
DB_USER = 'musthave'
DB_PASSWORD = 'tiger'
DB_NAME = 'musthave'